In [1]:
#a.
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.utils import to_categorical

# Load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Load the VGG16 model without the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Create data generators for data augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_test, y_test), epochs=1, callbacks=[early_stopping])

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 338s 215ms/step - accuracy: 0.4740 - loss: 3.2442 - val_accuracy: 0.6164 - val_loss: 1.1291
313/313 ━━━━━━━━━━━━━━━━━━━━ 53s 169ms/step - accuracy: 0.6166 - loss: 1.1322
Test Accuracy: 61.64%


b. ImageNet Classification with Deep Residual networks (ResNet)

In [5]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

# Load an image file that contains an image to be classified
img_path = '/kaggle/input/resnet50-indoor-object-detection/DATASET/test/Meja (354).jpg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add a batch dimension (since the model expects a batch of images)
x = np.expand_dims(x, axis=0)

# Preprocess the input image for the model
x = preprocess_input(x)

# Predict the class of the image
predictions = model.predict(x)

# Decode the top 3 predictions into human-readable class names
print('Predicted:', decode_predictions(predictions, top=3)[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: [('n02727426', 'apiary', 0.44437236), ('n03992509', "potter's_wheel", 0.1255737), ('n02797295', 'barrow', 0.119712114)]


c. Classifying Species of Flowers using Transfer Learning

In [6]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

# Load an image file that contains an image to be classified
img_path = '/kaggle/input/flowers/flowers/astilbe/10091895024_a2ea04cda6_c.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add a batch dimension
x = np.expand_dims(x, axis=0)

# Preprocess the input image
x = preprocess_input(x)

# Predict the class of the image
predictions = model.predict(x)

# Decode and print the top 3 predicted classes with their probabilities
print('Predicted:', decode_predictions(predictions, top=3)[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: [('n11879895', 'rapeseed', 0.31405503), ('n04604644', 'worm_fence', 0.0876353), ('n02793495', 'barn', 0.06969871)]
